# TermGPT v001 explained

Video explanation: https://youtu.be/O4EmRi0_CI4

First, some imports:

In [ ]:
import openai  # to access openai api
import colorama # to make pretty colors with prints
import time # sleeps
from contexts import TERMINAL_COMMANDS # the starting "prompt/context," more on this in the code
import subprocess # to issue commands and grab output (more work needed here w/ errors)
import re # for regular expressions
import requests # for web parsing
from bs4 import BeautifulSoup # for web parsing

Then some constants

In [ ]:
# load and set our key
openai.api_key = open("key.txt", "r").read().strip("\n")


# adds a few more prints, mainly prints out the full context in purple.
DEBUG = True  
# A context with a 1-shot example of how to behave, more on this later.
message_history = TERMINAL_COMMANDS  

# Patterns to detect web/file-to-read input 
READ_RE_PATTERN = r"--r \[(.*?)\]"
WEB_RE_PATTERN = r"--w \[(.*?)\]"

Next, we'll define the function for querying GPT-4

In [ ]:
def gpt_query(message_history, model="gpt-4", max_retries=15, sleep_time=2):
    retries = 0

    # if DEBUG, print out the full message history (helps to confirm exactly what GPT-4 is seeing)
    if DEBUG:
        print(colorama.Fore.MAGENTA + colorama.Style.DIM + "Message History: " + str(message_history) + colorama.Style.RESET_ALL)
    
    # Keep retrying for some number of tries, often the openai api just rejects from load.
    while retries < max_retries:
        try:

            # send chat context to openai API. 
            # see: https://github.com/Sentdex/ChatGPT-API-Basics for more details of how the API works.
            completion = openai.ChatCompletion.create(
                model=model,
                messages=message_history
            )

            # split out just the content of the reply and return it.
            reply_content = completion.choices[0].message.content
            if reply_content:
                return reply_content
        except Exception as e:
            print(colorama.Fore.YELLOW + colorama.Style.DIM + "Error during gpt_query() " + str(e) + colorama.Style.RESET_ALL)
            retries += 1
            time.sleep(sleep_time)

    # if we fail more than, say, 15 times, chances are we've got some other bug going on.
    raise Exception("Maximum retries exceeded. Check your code for errors.")

A simple paragraph text parser:

In [ ]:

# Mostly just used to exemplify TermGPT's ability to modify some existing script. 
# in this case, TermGPT wrote this to implement the web reading feature.
def extract_paragraphs(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = ''
    for para in soup.find_all('p'):
        paragraphs += para.text + '\n\n'
    return paragraphs

Print out a simple welcome message when script starts:

In [ ]:
for _ in range(3): print()
print(f"~{colorama.Style.BRIGHT}\033[4mWelcome to TermGPT{colorama.Style.RESET_ALL}~")
for _ in range(3): print()

print("The goal for TermGPT is to make rapid prototyping with GPT models even quicker and more fluid.")
print()


# Save console outputs for things like errors/warnings (doesnt fully work yet)
command_outputs = ""

Begin main loop:

In [ ]:
while True:
    GPT_DONE = False
    # prints out info on avail commands and then asks for user input
    print(colorama.Fore.CYAN + colorama.Style.BRIGHT + 'It looks like there is no current input. Please provide your objectives. \n"' 
+ colorama.Fore.YELLOW + '--c' + colorama.Fore.CYAN + '" to clear contextual history. \n"' 
+ colorama.Fore.YELLOW + '--o' + colorama.Fore.CYAN + '" to parse previous console command outputs and suggest further commands (can be helpful for big errors)\n"' 
+ colorama.Fore.YELLOW + '--w' + colorama.Fore.CYAN + '" to parse a website into context \n"' 
+ colorama.Fore.YELLOW + '--r [path/to/file.py]' + colorama.Fore.CYAN + '" to open and read some file into context (can do multiple files, and is embedded into some prompt.) Example: "Please change --r [mplexample.py] to be a dark theme"'+ colorama.Style.RESET_ALL)

    init_input = input(colorama.Fore.GREEN + colorama.Style.BRIGHT + "Command: " + colorama.Style.RESET_ALL)

First, a quick check to see if the user requested a clear, which will clear the context history:

In [ ]:
    if init_input.lower() == "clear"  or init_input.lower() == "--c":
        message_history = TERMINAL_COMMANDS
        command_outputs = ""
        print(colorama.Fore.YELLOW + colorama.Style.BRIGHT + "Context reset.\n\n" + colorama.Style.RESET_ALL)
        continue

Next, check to see if the input should be the previous console output (not fully functioning)

In [ ]:
    elif init_input.lower() == "output" or init_input.lower() == "--o":
        user_input = "NEW STARTING INPUT: I received the following console outputs after running everything. Is there anything I should do to address any errors, warnings, or notifications?: \n" + command_outputs


Otherwise, check to see if we have any websites or files to reach by seeing if we have any regex matches:

In [ ]:

    else:
        matches = re.findall(READ_RE_PATTERN, init_input)
        match_content = {}
        if matches:
            for n, match in enumerate(matches):
                with open(match, "r") as f:
                    match_content[match] = f.read()

        web_matches = re.findall(WEB_RE_PATTERN, init_input)
        web_match_content = {}
        if web_matches:
            for n, web_match in enumerate(web_matches):
                web_match_content[web_match] = extract_paragraphs(web_match)

If there are any matches, then we want to populate the content from the match into the input we send to GPT-4:

In [ ]:
        for match in match_content:
            original = f"--r [{match}]"
            replacement = f"\n file: {match}\n{match_content[match]}\n"
            init_input = init_input.replace(original, replacement)

        for web_match in web_match_content:
            original = f"--w [{web_match}]"
            replacement = f"\n website content: {web_match}\n{web_match_content[web_match]}\n"
            init_input = init_input.replace(original, replacement)

Then we'll add "NEW STARTING INPUT" to the front of this new input, to match an example shown to GPT-4 for what responses should look like, given some input. 

In [ ]:
        user_input = "NEW STARTING INPUT: " + init_input

Finally, we'll print the user input as a debug message if we have debug on, then we'll append the user_input to the message history, and start an empty list for `commands`, which is where we will store the commands suggested to us by GPT-4.



In [ ]:
    if DEBUG:
        print()
        print()
        print("User Input:")
        print()
        print(colorama.Fore.MAGENTA + colorama.Style.DIM + user_input + colorama.Style.RESET_ALL)
        print()
        print()

    message_history.append({"role": "user", "content": user_input})

    commands = []


Next, we begin allowing GPT-4 to iterate through proposed terminal commands to execute, so we'll continue until GPT-4 believes it's done.

But first, we should talk about how we actually get to this point/capability with GPT-4, so let's check out the pre-prompt that I'm using:


`contexts.py`, which is where `TERMINAL_COMMANDS` comes from:


```
TERMINAL_COMMANDS = [  
{"role": "user", "content": 'Given text input from a user looking to do programming, help to build a series of console commands that will achieve the goals of the prompt. Your responses will be one command at a time, where each response is JUST the command and nothing else. When ready for the next command, the user will say "NEXT" and you will respond with the next command. When we have reached the end of console commands, respond with "DONE" and then you will do it again with some new input from the user. If you understand, say "OK."'},
{"role": "assistant", "content": "OK."},
{"role": "user", "content": "NEW STARTING INPUT: show me a basic flask web dev example with templates?"},
{"role": "assistant", "content": "pip install flask"},
{"role": "user", "content": "NEXT"},
{"role": "assistant", "content": "mkdir basic_flask_app"},
{"role": "user", "content": "NEXT"},
{"role": "assistant", "content": '''cat << 'EOF' > basic_flask_app/app.py
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/about')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    app.run(debug=True)
EOF'''},
{"role": "user", "content": "NEXT"},

{"role": "assistant", "content": "mkdir basic_flask_app/templates"},
{"role": "user", "content": "NEXT"},


{"role": "assistant", "content": '''cat << EOF > basic_flask_app/templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>My Flask App</title>
</head>
<body>
    <h1>Welcome to My Flask App</h1>
    <p>This is the home page!</p>
</body>
</html>
EOF'''},
{"role": "user", "content": "NEXT"},


{"role": "assistant", "content": '''cat << 'EOF' > basic_flask_app/templates/about.html
<!DOCTYPE html>
<html>
<head>
    <title>About My Flask App</title>
</head>
<body>
    <h1>About My Flask App</h1>
    <p>This is a simple Flask app that demonstrates the use of templates!</p>
</body>
</html>
EOF
'''},
{"role": "user", "content": "NEXT"},
{"role": "assistant", "content": "python basic_flask_app/app.py"},
{"role": "user", "content": "NEXT"},
{"role": "assistant", "content": "DONE"}]```

What we have here is a pre-prompt that's a one-shot example of how I want GPT-4 to behave going forward, where a new starting input is presented, which contains something a user wants to do, and then GPT-4 is tasked with responding with just terminal commands, one at a time, going to the next one when the user says `"NEXT"`. If there are no more terminal commands to be issued, then GPT-4 replies with a `"DONE"`. In the one shot example, I show GPT-4 how I would like it to respond and give some examples of terminal commands to use, including naming/creating files and directories, writing code, and then running the code. This is where all the power of something like TermGPT comes from. Being all natural language, there are essentially endless prompts and structures that you might come up with to create even more unique outputs. For example, rather than using --w or --r, we could instead have a preprompt that automatically understands if we're talking about some file to be read, to just open and read it. The main concern here is that we still need some way to inject contents of files into the context, but we could still have GPT-4 notify us that it detected a read, then we still have logic to actually read the contents of the file into context. 

The overall objective is to make things easier and more natural for the user, so this is definitely the way, but I am still undecided on how I want to implement it yet. 

Coming back to the code for TermGPT, we are going to keep iterating until we reach that `"DONE"` response from GPT-4, each step of the way saving the proposed commands to a list of commands and saying `"NEXT"` back to GPT-4.

In [ ]:
    while not GPT_DONE:
        print(colorama.Fore.GREEN + colorama.Style.DIM + "Querying GPT for next command (these are not running yet)..." + colorama.Style.RESET_ALL)
        reply_content = gpt_query(message_history=message_history, model="gpt-4")

        message_history.append({"role": "assistant", "content": reply_content})
        message_history.append({"role": "user", "content": "NEXT"})
        print(colorama.Fore.WHITE + colorama.Style.DIM + reply_content + colorama.Style.RESET_ALL)

        if reply_content.lower() == "done" or reply_content.lower() == "done.":
            GPT_DONE = True
            print(colorama.Fore.YELLOW + colorama.Style.BRIGHT + "Done reached." + colorama.Style.RESET_ALL)
            break
        else:
            commands.append(reply_content)
        time.sleep(1.5)

Once we've reached a `"DONE"` from GPT-4, we will have a list of proposed terminal commands to run. It'd be unwise to blindly run these, so let's output them in red to make it clear what GPT-4 wants to do:

In [ ]:
    print(colorama.Fore.CYAN + colorama.Style.BRIGHT + "Proposed Commands:" + colorama.Style.RESET_ALL)
    for n, command in enumerate(commands):
        print(colorama.Fore.WHITE + colorama.Style.BRIGHT + "-"*5 + "Command "+ str(n)+ "-"*5 + colorama.Style.RESET_ALL)
        print()
        print(colorama.Fore.RED + colorama.Style.BRIGHT + command + colorama.Style.RESET_ALL)
        print()

After this, we're ready to run, or not, the commands:

In [ ]:
    if len(commands) > 0:
        print(colorama.Fore.CYAN + colorama.Style.BRIGHT + "Would you like to run these commands? " + colorama.Fore.YELLOW + "Read carefully!" + colorama.Style.RESET_ALL + colorama.Fore.CYAN + colorama.Style.BRIGHT + " Do not run if you don't understand and want the outcomes. (y/n)" + colorama.Style.RESET_ALL)

        run_commands = input("Run the commands? (y/n): ")
        if run_commands.lower() == "y":
            for n, command in enumerate(commands):
                try:
                    print(colorama.Fore.CYAN + colorama.Style.BRIGHT + "Running command " + str(n) + ": " + command + colorama.Style.RESET_ALL)
                    output = subprocess.check_output(command, shell=True, text=True).strip()
                    
                except subprocess.CalledProcessError as e:
                    print(colorama.Fore.YELLOW + colorama.Style.DIM + "Error during command execution: " + str(e) + colorama.Style.RESET_ALL)
                    output = e.output.strip()

                print(output)
                command_outputs += output + "\n\n"
        else:
            print(colorama.Fore.CYAN + colorama.Style.BRIGHT + "Okay, not running commands." + colorama.Style.RESET_ALL)

And that's all there is to it so far. I think a lot more can be done with natural language and a powerful LLM here. Rather than building out too much further with GPT-4 though, I would like to implement this with an open source model instead, and ideally one with a more permissive and clear license than the LLaMA variants. I found many differences even between GPT-4 and GPT-3.5 in terms of what works and doesn't, so I'd rather settle more on a model before going too much further, so I will probably be working on trying to find an open source model that works well for this next, then building more features on top of that.